<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Задача этого проекта - обучить модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности.

Целевое значением метрики качества *F1* не меньше 0.75

## Подготовка

In [1]:
!pip install spacy -q

In [2]:
import numpy as np
import pandas as pd
#учет времени выполнения
import time
from tqdm.notebook import tqdm
tqdm.pandas()
# Инструменты подбора гиперпараметров
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
# пайплайны
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
# Модели
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier
# метрики
from sklearn.metrics import f1_score
#обработка текстов
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from pymystem3 import Mystem
from sklearn.feature_extraction.text import CountVectorizer
from lightgbm import LGBMClassifier
import warnings

import spacy
spcy = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

RANDOM_STATE = 88

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
data = pd.read_csv('/datasets/toxic_comments.csv')
display(data.info())
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


None

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
data = data.drop('Unnamed: 0', axis=1)

Очистим текст от элементов разметки и прочего мусора:

In [5]:
data['clean_text'] = data.text.transform(lambda x: re.sub(r'\W+|\d+|_+', ' ', x))
data['clean_text'] = data.clean_text.transform(lambda x: re.sub(r'\s{2,}', ' ', x))

Выполним лемматизацию:

In [6]:
lemm_texts = []
total = data.shape[0]
nlp_pipe = spcy.pipe(data.clean_text.values, disable = ['ner', 'parser'])

start = time.time()

for doc in tqdm(nlp_pipe, total=total):
    lemm_text = " ".join([token.lemma_ for token in doc])    
    lemm_texts.append(lemm_text) 

data['lemm_text'] = lemm_texts

finish = time.time()
exec_time = int((finish - start) // 60)
print(f'Время выполнения лемматизации {exec_time} минут')

  0%|          | 0/159292 [00:00<?, ?it/s]

Время выполнения лемматизации 10 минут


In [7]:
data = data[['lemm_text', 'toxic']]

In [9]:
train, test = train_test_split(data, test_size = 0.2)
X_train, y_train = train.drop('toxic', axis=1), train.toxic
X_test, y_test = test.drop('toxic', axis=1), test.toxic

In [10]:
corpus_train = X_train['lemm_text'].values

In [11]:
pipe_train =  Pipeline([
    ('vectorizer', CountVectorizer(stop_words='english')),
    ('models', 'passthrough')
])

In [12]:
param_grid = [
    # словарь для модели DecisionTreeClassifier()
    {
        'models': [DecisionTreeClassifier(random_state=RANDOM_STATE)],
        'models__min_samples_split': range(2, 5),
         'models__min_samples_leaf': range(2, 5),
         'models__max_depth': range(2, 5),
        'models__max_features': range(2, 5)
    },
    
    # словарь для модели LogisticRegression()
    {
        'models': [LogisticRegression(random_state=RANDOM_STATE)],
        'models__penalty': ['l1','l2','elasticnet', None],
        'models__C': range(1, 5)
    },
    
    # словарь для модели KNeighborsClassifier()
    {
        'models': [KNeighborsClassifier()],
        'models__n_neighbors': range(2, 10)
    },
    
    # словарь для модели LGBMClassifier()
    {
        'models': [LGBMClassifier()],
    },
]

In [13]:
cv_train = RandomizedSearchCV(
    pipe_train,
    param_grid,
    n_jobs=1,
    cv=5,
    n_iter=10,
    scoring='f1'
)

## Обучение

In [14]:
warnings.filterwarnings('ignore')
cv_train.fit(corpus_train, y_train)

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('vectorizer',
                                              CountVectorizer(stop_words='english')),
                                             ('models', 'passthrough')]),
                   n_jobs=1,
                   param_distributions=[{'models': [DecisionTreeClassifier(random_state=88)],
                                         'models__max_depth': range(2, 5),
                                         'models__max_features': range(2, 5),
                                         'models__min_samples_leaf': range(2, 5),
                                         'models__min_samples_split': range(2, 5)},
                                        {'models': [LogisticRegression(random_state=88)],
                                         'models__C': range(1, 5),
                                         'models__penalty': ['l1', 'l2',
                                                             'elasticnet',
                                                             None]},
                                        {'models': [KNeighborsClassifier()],
                                         'models__n_neighbors': range(2, 10)},
                                        {'models': [LGBMClassifier()]}],
                   scoring='f1')

In [15]:
pd.set_option('display.max_colwidth', None)
result = pd.DataFrame(cv_train.cv_results_)
result[
    ['rank_test_score', 'params']
].sort_values('rank_test_score').head(5)

,rank_test_score,params
6,1,"{'models__penalty': None, 'models__C': 1, 'models': LogisticRegression(random_state=88)}"
5,2,"{'models__n_neighbors': 3, 'models': KNeighborsClassifier()}"
0,3,"{'models__min_samples_split': 4, 'models__min_samples_leaf': 2, 'models__max_features': 3, 'models__max_depth': 3, 'models': DecisionTreeClassifier(random_state=88)}"
1,3,"{'models__min_samples_split': 2, 'models__min_samples_leaf': 4, 'models__max_features': 4, 'models__max_depth': 3, 'models': DecisionTreeClassifier(random_state=88)}"
2,3,"{'models__min_samples_split': 4, 'models__min_samples_leaf': 4, 'models__max_features': 2, 'models__max_depth': 4, 'models': DecisionTreeClassifier(random_state=88)}"


In [16]:
warnings.filterwarnings('ignore')
best_suite = cv_train.best_estimator_
best_suite.fit(corpus_train, y_train)

Pipeline(steps=[('vectorizer', CountVectorizer(stop_words='english')),
                ('models',
                 LogisticRegression(C=1, penalty=None, random_state=88))])

In [17]:
corpus_test = X_test['lemm_text'].values
y_pred = best_suite.predict(corpus_test)
f1 = f1_score(y_test, y_pred)
print(f'F1-мера обученной модели на тестовых данных: {round(f1,2)}')

F1-мера обученной модели на тестовых данных: 0.75


## Выводы

В процессе работы над проектом мы выполнили следующие шаги:
* Предобработка данных
* Трансформация данных для обучения
* Настройка пайплайнов
* Подбор лучшей модели и ее гиперпараметров
* Обучение лучшей модели и проверка ее на тестовых данных

В процессе предобработки данных, мы очистили датасет от ненужных данных, а также почистили текст от спецсимволов и другого мусора.

Обработанные таким образом данные мы подвергли лемматизации при помощи библиотеки spaCy, после чего преобразовали в корпус слов для передачи в пайплайн обучения.

При построении пайплайна был добавлен шаг векторизации входящего корпуса слов при помощи инструмента CountVectorizer.

Для подбора гиперпараметров пайплайну были переданы несколько моделей:
- DecisionTreeClassifier
- LogisticRegression
- KNeighborsClassifier
- LGBMClassifier

Подбор гиперпараметров выполнялся методом Randomized Search с кросс-валидацией.

Лучшая модель по результатам подбора гиперпараметров показала значение контрольной метрики F1 на тестовых данных выше целевого порога 0,75 согласно требованиям.